In [3]:
import numpy as np
import random

HIS_tuple.txt format: indexes of dis, sym, herb

In [4]:
with open('../data/HIS_tuple.txt') as dis_dct_file:
    lines = dis_dct_file.readlines()
lines = [line.strip().split() for line in lines]
training = []
for line in lines:
    if len(line) == 3:
        record = [list(map(int, entry.strip()[:-1].split(':'))) for entry in line]
        training.append(record)

In [5]:
# Helper function that parses a text file into a dictionary
def parse_dict(file_name): 
    with open('../data/'+file_name) as file:
        lines = file.readlines()
    lines = [line.strip().split() for line in lines]
    dict = {}
    for line in lines:
        if len(line) == 2:
            dict[int(line[1])] = line[0]
    return dict

In [6]:
sym_dict = parse_dict('sym_dct.txt')
herb_dict = parse_dict('herb_dct.txt')
dis_dict = parse_dict('dis_dct.txt')

In [7]:
num_sym = len(sym_dict)
num_herb = len(herb_dict)
num_dis = len(dis_dict)
num_record = len(training)
print("Number of symptoms: ", num_sym)
print("Number of herbs: ", num_herb)
print("Number of diseases: ", num_dis)
print("Number of records: ",num_record)

Number of symptoms:  15070
Number of herbs:  848
Number of diseases:  1558
Number of records:  9486


In [8]:
count = 0
for record in training:
    count += len(record[1])*len(record[2])
count*num_dis

3040217322

In [9]:
# Initialize symptom popularity dictionary
sym_dict = {}
for i in range(num_sym):
    sym_dict[i+1] = 0

# Initialize herb popularity dictionary
herb_dict = {}
for i in range(num_herb):
    herb_dict[i+1] = 0
    
# Initialize disease popularity dictionary
dis_dict = {}
for i in range(num_dis):
    dis_dict[i+1] = 0

# Loop through training data to fill in the popularity dictionaries
for record in training[:1000]:
    for disease in record[0]:
        dis_dict[disease] += 1
    for symptom in record[1]:
        sym_dict[symptom] += 1
    for herb in record[2]:
        herb_dict[herb] += 1
    

In [16]:
print(dis_dict[2])

746

In [9]:
# Sort each popularity dictionary based on count
sym_list = [(k, sym_dict[k]) for k in sorted(sym_dict, key=sym_dict.get, reverse=True)]
dis_list = [(k, dis_dict[k]) for k in sorted(dis_dict, key=dis_dict.get, reverse=True)]
herb_list = [(k, herb_dict[k]) for k in sorted(herb_dict, key=herb_dict.get, reverse=True)]

# Extract the most popular symptoms, herbs, and diseases based on popularity
key_sym = []
key_herb = []
key_dis = []
# Get top 500 popular symptoms and herbs
for i in range(500):
    key_sym.append(sym_list[i][0])
    key_herb.append(herb_list[i][0])
# Get top 200 popular diseases
for i in range(92):
    key_dis.append(dis_list[i][0])

In [10]:
print(dis_list)

[(1, 375), (3, 76), (6, 61), (4, 60), (5, 55), (7, 48), (2, 44), (14, 43), (8, 38), (13, 37), (17, 32), (21, 32), (52, 32), (15, 26), (26, 26), (20, 22), (9, 21), (10, 21), (19, 20), (25, 20), (30, 20), (24, 19), (44, 18), (31, 16), (55, 16), (22, 15), (11, 14), (29, 14), (36, 14), (48, 13), (62, 13), (84, 13), (16, 12), (40, 12), (41, 12), (74, 12), (12, 11), (34, 11), (59, 11), (32, 10), (33, 10), (39, 10), (50, 10), (91, 10), (27, 8), (42, 8), (18, 7), (28, 7), (45, 7), (57, 7), (61, 7), (75, 7), (81, 7), (95, 7), (54, 6), (58, 6), (71, 6), (77, 6), (23, 5), (37, 5), (51, 5), (56, 5), (65, 5), (72, 5), (76, 5), (83, 5), (96, 5), (35, 4), (43, 4), (53, 4), (64, 4), (68, 4), (82, 4), (49, 3), (66, 3), (67, 3), (80, 3), (89, 3), (94, 3), (38, 2), (47, 2), (69, 2), (70, 2), (78, 2), (86, 2), (88, 2), (46, 1), (60, 1), (73, 1), (79, 1), (85, 1), (92, 1), (63, 0), (87, 0), (90, 0), (93, 0), (97, 0), (98, 0), (99, 0), (100, 0), (101, 0), (102, 0), (103, 0), (104, 0), (105, 0), (106, 0), (1

In [11]:
# Process the training data, eliminate rare diseases, symptoms, and herbs from the training records
key_dis_set = set(key_dis)
key_sym_set = set(key_sym)
key_dis_set = set(key_dis)

key_training = []
for record in training[:1000]:
    new_dis = [x for x in record[0] if x in key_dis_set]
    if len(new_dis) == 0:
        continue
    new_sym = [x for x in record[1] if x in key_sym_set]
    if len(new_sym) == 0:
        continue
    new_herb = [x for x in record[2] if x in key_dis_set]
    if len(new_herb) == 0:
        continue
    key_training.append([new_dis, new_sym, new_herb])
    
len(key_training)

985

In [12]:
tsh_tuples = []
for i in range(len(key_training)):
    record = key_training[i]
    for symptom in record[1]:
        for herb in record[2]:
            tsh_tuples.append((i, symptom, herb))
len(tsh_tuples)

154953

In [13]:
key_num_dis = len(key_dis)
key_num_sym = len(key_sym)
key_num_herb = len(key_herb)
key_num_record = len(key_training)
print("number of diseases: ", key_num_dis)
print("number of symtom: ", key_num_sym)
print("number of herb: ", key_num_herb)
print("number of record: ", key_num_record)

number of diseases:  92
number of symtom:  500
number of herb:  500
number of record:  985


EM to train the model.
At each step, random pick 10 records as the training data,
then iterator until model converges

In [14]:
def normalizer(input_dict):
    small_mass = 10**-20
    normalizer = 0.0
    for key in input_dict.keys():
        normalizer += input_dict[key]
    normalizer += len(input_dict) * small_mass
    for key in input_dict.keys():
        input_dict[key] = (input_dict[key] + small_mass) / normalizer

In [15]:
# Initialize the model

# P(d)
P_d = {}
for d in key_dis:
    P_d[d] = random.randint(1,3)
normalizer(P_d)

# P(t|d)
P_t_d = {}
for d in key_dis:
    t_dict = {}
    for t in range(len(key_training)):
        t_dict[t] = random.randint(1,3)
    normalizer(t_dict)
    P_t_d[d] = t_dict

# P(s|d)
P_s_d = {}
for d in key_dis:
    s_dict = {}
    for s in key_sym:
        s_dict[s] = random.randint(1,3)
    normalizer(s_dict)
    P_s_d[d] = s_dict

# P(h|d)
P_h_d = {}
for d in key_dis:
    h_dict = {}
    for h in key_herb:
        h_dict[h] = random.randint(1,3)
    normalizer(h_dict)
    P_h_d[d] = h_dict

### EM to train the model

In [26]:
for i in range(5):
    # Compute P(d|s,h,t)
    # Declare a small number added to each entry to prevent 0 probability
    P_d_sht = {}
    for i in range(len(tsh_tuples)):
        (t,s,h) = tsh_tuples[i]
        prob_list = {}
        for d in key_dis:
            prob_list[d] = P_d[d] * P_t_d[d][t] * P_s_d[d][s] * P_h_d[d][h]
        normalizer(prob_list)
        P_d_sht[(t,s,h)] = prob_list

    old_P_d = np.asarray(list(P_d.values()))

    # Clear out the old distributions from the last iteration
    P_d = dict.fromkeys(P_d, 0.0)
    for d in key_dis:
        P_t_d[d] = dict.fromkeys(P_t_d[d], 0)
        P_s_d[d] = dict.fromkeys(P_s_d[d], 0)
        P_h_d[d] = dict.fromkeys(P_h_d[d], 0)
    # Perform the M step
    for (t,s,h) in P_d_sht.keys():
        d_dict = P_d_sht[(t,s,h)]
        for d in d_dict.keys():
            # update P(d)
            P_d[d] += d_dict[d]
            # update P(t|d)
            P_t_d[d][t] += d_dict[d]
            # update P_s_d
            P_s_d[d][s] += d_dict[d]
            # update P_h_d
            P_h_d[d][h] += d_dict[d]

    # Normalize the new distribution
    normalizer(P_d)
    for d in key_dis:
        normalizer(P_t_d[d])
        normalizer(P_s_d[d])
        normalizer(P_h_d[d])

    new_P_d = np.asarray(list(P_d.values()))
    print(np.abs(old_P_d - new_P_d).max())

9.031275372017866e-05
8.407273167544173e-05
8.26769563180424e-05
9.736791586057239e-05
7.604311895236839e-05


In [27]:
# Pickle file for further usage
import pickle

In [28]:
pickle.dump( P_d, open( "../training_result/P_d.p", "wb" ) )
pickle.dump( P_t_d, open( "../training_result/P_t_d.p", "wb" ) )
pickle.dump( P_s_d, open( "../training_result/P_s_d.p", "wb" ) )
pickle.dump( P_h_d, open( "../training_result/P_h_d.p", "wb" ) )

In [29]:
pickle.dump(key_dis, open("../training_result/key_dis", "wb"))
pickle.dump(key_sym, open("../training_result/key_sym", "wb"))
pickle.dump(key_herb, open("../training_result/key_herb", "wb"))

In [38]:
print(key_training)

[[[1], [12, 18, 75, 355, 8, 33, 152, 14, 22, 2], [9, 4, 31, 5, 20, 17, 8, 12, 2, 33, 23, 22, 1]], [[94], [6], [13, 6, 2, 10, 7, 21, 5, 14, 8, 23, 3, 1, 9, 31, 18, 17, 28, 20, 26, 24, 12, 91, 74]], [[31], [59, 19, 123, 17, 26, 166, 97, 144, 32, 16, 8, 60, 29, 336, 13, 2], [21, 9, 4, 12, 29, 20, 15, 75, 19, 14, 8, 55, 45, 2, 77, 11, 7]], [[2], [59, 29, 153, 17, 13, 54, 166, 97, 32, 16, 8, 77, 60, 483, 336, 2], [4, 92, 75, 55, 2, 7, 21, 8, 14, 11, 9, 15, 19, 17, 33, 34, 12, 29, 77]], [[2], [21, 26, 75, 9, 3, 42, 5, 25, 70, 1], [21, 5, 4, 23, 31, 71, 2, 37, 36, 52, 32, 8, 58, 74, 20, 33, 26, 27, 3]], [[14], [131, 60, 14, 19, 196, 113, 31, 26, 142, 68, 188, 12, 21, 40, 28, 129, 16, 1117, 25, 10], [21, 5, 4, 23, 31, 71, 2, 37, 36, 52, 32, 8, 58, 74, 20, 33, 26, 27, 3]], [[10], [131, 60, 14, 19, 196, 113, 31, 26, 142, 68, 188, 12, 21, 40, 28, 129, 16, 1117, 25, 10], [75, 28, 6, 39, 2, 56, 3, 86, 21, 43, 8, 7, 47, 27, 11, 1, 9, 31, 17, 32, 76, 16, 77]], [[1, 10], [6], [75, 28, 6, 39, 2, 56, 3,

In [25]:
print(key_dis)

[1, 3, 6, 4, 5, 7, 2, 14, 8, 13, 17, 21, 52, 15, 26, 20, 9, 10, 19, 25, 30, 24, 44, 31, 55, 22, 11, 29, 36, 48, 62, 84, 16, 40, 41, 74, 12, 34, 59, 32, 33, 39, 50, 91, 27, 42, 18, 28, 45, 57, 61, 75, 81, 95, 54, 58, 71, 77, 23, 37, 51, 56, 65, 72, 76, 83, 96, 35, 43, 53, 64, 68, 82, 49, 66, 67, 80, 89, 94, 38, 47, 69, 70, 78, 86, 88, 46, 60, 73, 79, 85, 92]
